In [1]:
import os
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/HIV_inhibitors_classification_and_generation/research'

In [2]:
os.chdir("../")
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/HIV_inhibitors_classification_and_generation'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_csv: Path
    train_csv: Path
    test_csv: Path
    train_folder: Path
    test_folder: Path
    params: dict
    dataset_val_status: bool

In [4]:
from hivclass.constants import *
from hivclass.utils.main_utils import create_directories, read_yaml

class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        params = self.params.data_transformation
        dataset_val_status_file = self.config.data_validation.STATUS_FILE
        
        with open(dataset_val_status_file, 'r') as f:
            status = f.read()
        
        status = bool(str.split(status)[-1])
        
        create_directories([config.root_dir, config.train_folder, config.test_folder])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_csv=config.data_csv,
            train_csv=config.train_csv,
            test_csv=config.test_csv,
            train_folder=config.train_folder,
            test_folder=config.test_folder,
            params=params.split_size,
            dataset_val_status=status
        )
        
        return data_transformation_config

In [5]:
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import numpy as np
from rdkit import Chem
import torch
import torch_geometric
from torch_geometric.data import Dataset, Data
import deepchem as dc

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
    
    def test_split_train_balanced(self):
        df = pd.read_csv(self.config.data_csv)
        
        # Separate positive and negative cases
        p_val = df[df.HIV_active == 1].to_numpy()
        n_val = df[df.HIV_active == 0].to_numpy()

        # Ensure class balance by selecting the smaller group as the target
        if len(p_val) >= len(n_val):
            big, small = p_val, n_val
        else:
            big, small = n_val, p_val

        # Stratified test split
        small_train, small_test = train_test_split(
            small,
            test_size=self.config.params,
            random_state=42
        )
        
        big_train, big_test = train_test_split(
            big,
            test_size=(self.config.params * len(small) / len(big)),
            random_state=42
        )

        test = np.concatenate([small_test, big_test])
        
        # Ensure the train set remains balanced by oversampling the smaller class
        train = np.concatenate([
            big_train,
            random.choices(small_train, k=len(big_train) - len(small_train))
        ])

        # Convert back to DataFrame
        train_df = pd.DataFrame(train, columns=df.columns).sample(frac=1, random_state=42)
        train_df.reset_index(drop=True, inplace=True)
        train_df.to_csv(self.config.train_csv, index=False)
        
        test_df = pd.DataFrame(test, columns=df.columns).sample(frac=1, random_state=42)
        test_df.reset_index(drop=True, inplace=True)
        test_df.to_csv(self.config.test_csv, index=False)
    
    def data_preparation(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"

        datas = [self.config.train_csv, self.config.test_csv]
        save_dirs = [self.config.train_folder, self.config.test_folder]
        featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)

        all_rdkit_data, all_deepchem_data = [], []

        for data, save_dir in zip(datas, save_dirs):
            data_df = pd.read_csv(data)
            name = os.path.splitext(os.path.basename(data))[0]  # Extract filename without extension

            for mol in tqdm(data_df.itertuples(index=True), total=len(data_df), desc=f"Processing {name}"):

                mol_obj = Chem.MolFromSmiles(mol.smiles)
                if mol_obj is None:
                    continue  # Skip invalid SMILES strings
                
                label = torch.tensor(mol.HIV_active, dtype=torch.int64, device=device)
                
                # # Extract molecular graph features
                # node_features = self.get_node_features(mol_obj, device)
                # edge_features = self.get_edge_features(mol_obj, device)
                # adjacency_matrix = self.get_adjacency_matrix(mol_obj, device)

                # # Create PyTorch Geometric Data objects
                # data_rdkit = Data(
                #     x=node_features, edge_attr=edge_features,
                #     edge_index=adjacency_matrix, y=label, smiles=mol.smiles
                # )
                # all_rdkit_data.append(data_rdkit)
                
                graph_features = featurizer._featurize(mol_obj)

                data_deepchem = Data(
                    x=torch.tensor(graph_features.node_features, dtype=torch.float, device=device),
                    edge_attr=torch.tensor(graph_features.edge_features, dtype=torch.float, device=device),
                    edge_index=torch.tensor(graph_features.edge_index, dtype=torch.long, device=device),
                    y=label, smiles=mol.smiles
                )
                all_deepchem_data.append(data_deepchem)

                # Save preprocessed molecule graph
                torch.save(data_deepchem, os.path.join(save_dir, f"{name}_{mol[0]}.pt"))

    # def get_node_features(self, mol, device):
    #     all_atom_features = torch.tensor([
    #         [
    #             atom.GetAtomicNum(),
    #             atom.GetDegree(),
    #             atom.GetFormalCharge(),
    #             int(atom.GetHybridization()),  # Convert Hybridization enum to int
    #             atom.GetIsAromatic(),
    #             atom.GetTotalNumHs(),
    #             atom.GetNumRadicalElectrons(),
    #             atom.IsInRing(),
    #             int(atom.GetChiralTag())  # Convert ChiralTag enum to int
    #         ]
    #         for atom in mol.GetAtoms()
    #     ], dtype=torch.float, device=device)

    #     return all_atom_features
    
    # def get_edge_features(self, mol, device):
    #     all_bond_features = torch.tensor([
    #         [
    #             bond.GetBondTypeAsDouble(),
    #             bond.IsInRing(),
    #             bond.GetIsConjugated(),
    #             int(bond.GetStereo())  # Convert Stereo enum to integer
    #         ]
    #         for bond in mol.GetBonds()
    #     ], dtype=torch.float, device=device)

    #     return all_bond_features
    
    # def get_adjacency_matrix(self, mol, device):
    #     edge_indices = torch.tensor(
    #         [[bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()] for bond in mol.GetBonds()]
    #         + [[bond.GetEndAtomIdx(), bond.GetBeginAtomIdx()] for bond in mol.GetBonds()],
    #         dtype=torch.long, device=device
    #     ).T  # Transpose for proper shape (2, num_edges)

    #     return edge_indices
    
    def transformation_compose(self):
        self.test_split_train_balanced()
        self.data_preparation()

[2025-03-10 19:49:24,360: INFO: __init__: Enabling RDKit 2024.09.5 jupyter extensions]
[2025-03-10 19:49:32,303: WARNING: rdkit_descriptors: No normalization for SPS. Feature removed!]
[2025-03-10 19:49:32,304: WARNING: rdkit_descriptors: No normalization for AvgIpc. Feature removed!]
[2025-03-10 19:49:32,304: WARNING: rdkit_descriptors: No normalization for NumAmideBonds. Feature removed!]
[2025-03-10 19:49:32,305: WARNING: rdkit_descriptors: No normalization for NumAtomStereoCenters. Feature removed!]
[2025-03-10 19:49:32,306: WARNING: rdkit_descriptors: No normalization for NumBridgeheadAtoms. Feature removed!]
[2025-03-10 19:49:32,306: WARNING: rdkit_descriptors: No normalization for NumHeterocycles. Feature removed!]
[2025-03-10 19:49:32,307: WARNING: rdkit_descriptors: No normalization for NumSpiroAtoms. Feature removed!]
[2025-03-10 19:49:32,308: WARNING: rdkit_descriptors: No normalization for NumUnspecifiedAtomStereoCenters. Feature removed!]
[2025-03-10 19:49:32,308: WARNING:

In [6]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    results_rdkit, results_deepchem = data_transformation.transformation_compose()

except Exception as e:
    raise e

[2025-03-10 19:49:54,146: INFO: main_utils: created directory at: artifacts]
[2025-03-10 19:49:54,148: INFO: main_utils: created directory at: artifacts/data_transformation]
[2025-03-10 19:49:54,148: INFO: main_utils: created directory at: artifacts/data_transformation/train]
[2025-03-10 19:49:54,150: INFO: main_utils: created directory at: artifacts/data_transformation/test]


Processing train:   0%|          | 23/77636 [00:00<17:18, 74.74it/s] 

[2025-03-10 19:49:54,941: INFO: molecule_feature_utils: input S not in allowable set ['SP', 'SP2', 'SP3']:]
[2025-03-10 19:49:54,953: INFO: molecule_feature_utils: input UNSPECIFIED not in allowable set ['SP', 'SP2', 'SP3']:]
[2025-03-10 19:49:54,954: INFO: molecule_feature_utils: input UNSPECIFIED not in allowable set ['SP', 'SP2', 'SP3']:]


Processing train:   0%|          | 32/77636 [00:00<18:44, 69.00it/s]

[2025-03-10 19:49:55,011: INFO: molecule_feature_utils: input S not in allowable set ['SP', 'SP2', 'SP3']:]


...

[2025-03-10 20:12:54,685: INFO: molecule_feature_utils: input S not in allowable set ['SP', 'SP2', 'SP3']:]


Processing test:  98%|█████████▊| 564/578 [00:10<00:00, 52.42it/s]

[2025-03-10 20:12:54,921: INFO: molecule_feature_utils: input UNSPECIFIED not in allowable set ['SP', 'SP2', 'SP3']:]


Processing test: 100%|██████████| 578/578 [00:10<00:00, 56.99it/s]


TypeError: cannot unpack non-iterable NoneType object